In [1]:
from transformers import pipeline

c:\Suraj\projects\emailsentiment\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

In [3]:
print(sentiment_analysis("I love this!"))

[{'label': 'POSITIVE', 'score': 0.9988656044006348}]


In [4]:
import pandas as pd

file_path = r'data\dump'

data = pd.read_parquet(file_path)

print(f"data shape: {data.shape}")

data shape: (17967, 11)


In [5]:
data.head()

,id,subject,model_lable,score,emotion,coe_label,remark,cleaned_body,markedon,run_date,rn
0,65e7404b-a719-41fa-8a3b-05e10711ed01,RE: EXTERNAL MAILRe: EXTERNAL MAILRequesting f...,NEUTRAL,0.726611,gratitude,neutral,Submittal link provided,"Dear Puneet, kindly note that we shared all th...",2024-11-22 08:53:28.377,2024-11-18,1
1,171ac239-f351-4668-806e-9d91a9a08096,RE: Drafting from KPFF San Diego,NEUTRAL,0.748543,neutral,neutral,Client shared the information that they upload...,Hello Koushik! Markups for Las Plumas have bee...,2024-11-18 12:00:08.903,2024-11-18,1
2,01063faa-8a18-45ab-a336-3220c953cf68,Paragon - RFIs For Incorporation 023 - U-N-F-1...,NEUTRAL,0.544132,gratitude,neutral,Information,"Hello Hemen, We received finalized information...",2024-11-20 10:53:29.957,2024-11-18,1
3,a817a73a-0dea-445d-a148-875c15cd8e5c,Studio Work - 11-17-2024,NEUTRAL,0.811226,neutral,neutral,It was scope-related information from the client.,"Please work on the following in order, dont mo...",2024-11-22 05:50:59.463,2024-11-18,1
4,3e06a9e4-bd12-4100-bf08-d42d6c88e7c1,Re: TIT_0003: Centennial Yards - Hotel: Todays...,NEUTRAL,0.652679,gratitude,neutral,Client informed that he will not be attending ...,I will not be attending. Thanks Chuck Hannon S...,2024-11-19 11:55:59.597,2024-11-18,1


In [6]:
def truncate_to_bytes(text, max_bytes=510):
    encoded_text = text.encode('utf-8')  # Convert to bytes
    if len(encoded_text) > max_bytes:
        # Truncate the byte data and decode back to string
        truncated_text = encoded_text[:max_bytes].decode('utf-8', errors='ignore')
        return truncated_text
    return text

In [10]:
# Define a function to handle truncation based on character length
def predict_sentiment(text):
    # Truncate the text to approximately 2000 characters
    text_truc = truncate_to_bytes(text)
    return sentiment_analysis(text_truc)

In [ ]:
## 92m 51 sec
data['model_sentiment'] = data['cleaned_body'].apply(predict_sentiment)

In [12]:
data.head()

,id,subject,model_lable,score,emotion,coe_label,remark,cleaned_body,markedon,run_date,rn,model_sentiment
0,65e7404b-a719-41fa-8a3b-05e10711ed01,RE: EXTERNAL MAILRe: EXTERNAL MAILRequesting f...,NEUTRAL,0.726611,gratitude,neutral,Submittal link provided,"Dear Puneet, kindly note that we shared all th...",2024-11-22 08:53:28.377,2024-11-18,1,"[{'label': 'POSITIVE', 'score': 0.997077345848..."
1,171ac239-f351-4668-806e-9d91a9a08096,RE: Drafting from KPFF San Diego,NEUTRAL,0.748543,neutral,neutral,Client shared the information that they upload...,Hello Koushik! Markups for Las Plumas have bee...,2024-11-18 12:00:08.903,2024-11-18,1,"[{'label': 'POSITIVE', 'score': 0.994045376777..."
2,01063faa-8a18-45ab-a336-3220c953cf68,Paragon - RFIs For Incorporation 023 - U-N-F-1...,NEUTRAL,0.544132,gratitude,neutral,Information,"Hello Hemen, We received finalized information...",2024-11-20 10:53:29.957,2024-11-18,1,"[{'label': 'POSITIVE', 'score': 0.981047093868..."
3,a817a73a-0dea-445d-a148-875c15cd8e5c,Studio Work - 11-17-2024,NEUTRAL,0.811226,neutral,neutral,It was scope-related information from the client.,"Please work on the following in order, dont mo...",2024-11-22 05:50:59.463,2024-11-18,1,"[{'label': 'POSITIVE', 'score': 0.994972944259..."
4,3e06a9e4-bd12-4100-bf08-d42d6c88e7c1,Re: TIT_0003: Centennial Yards - Hotel: Todays...,NEUTRAL,0.652679,gratitude,neutral,Client informed that he will not be attending ...,I will not be attending. Thanks Chuck Hannon S...,2024-11-19 11:55:59.597,2024-11-18,1,"[{'label': 'NEGATIVE', 'score': 0.997994661331..."


In [13]:
data['model_lable_n'] = data['model_sentiment'].apply(lambda x: x[0]['label'])
data['model_score_n'] = data['model_sentiment'].apply(lambda x: x[0]['score'])

In [14]:
data.head()

,id,subject,model_lable,score,emotion,coe_label,remark,cleaned_body,markedon,run_date,rn,model_sentiment,model_lable_n,model_score_n
0,65e7404b-a719-41fa-8a3b-05e10711ed01,RE: EXTERNAL MAILRe: EXTERNAL MAILRequesting f...,NEUTRAL,0.726611,gratitude,neutral,Submittal link provided,"Dear Puneet, kindly note that we shared all th...",2024-11-22 08:53:28.377,2024-11-18,1,"[{'label': 'POSITIVE', 'score': 0.997077345848...",POSITIVE,0.997077
1,171ac239-f351-4668-806e-9d91a9a08096,RE: Drafting from KPFF San Diego,NEUTRAL,0.748543,neutral,neutral,Client shared the information that they upload...,Hello Koushik! Markups for Las Plumas have bee...,2024-11-18 12:00:08.903,2024-11-18,1,"[{'label': 'POSITIVE', 'score': 0.994045376777...",POSITIVE,0.994045
2,01063faa-8a18-45ab-a336-3220c953cf68,Paragon - RFIs For Incorporation 023 - U-N-F-1...,NEUTRAL,0.544132,gratitude,neutral,Information,"Hello Hemen, We received finalized information...",2024-11-20 10:53:29.957,2024-11-18,1,"[{'label': 'POSITIVE', 'score': 0.981047093868...",POSITIVE,0.981047
3,a817a73a-0dea-445d-a148-875c15cd8e5c,Studio Work - 11-17-2024,NEUTRAL,0.811226,neutral,neutral,It was scope-related information from the client.,"Please work on the following in order, dont mo...",2024-11-22 05:50:59.463,2024-11-18,1,"[{'label': 'POSITIVE', 'score': 0.994972944259...",POSITIVE,0.994973
4,3e06a9e4-bd12-4100-bf08-d42d6c88e7c1,Re: TIT_0003: Centennial Yards - Hotel: Todays...,NEUTRAL,0.652679,gratitude,neutral,Client informed that he will not be attending ...,I will not be attending. Thanks Chuck Hannon S...,2024-11-19 11:55:59.597,2024-11-18,1,"[{'label': 'NEGATIVE', 'score': 0.997994661331...",NEGATIVE,0.997995


In [15]:
excel_file = r"data/output_file_sentiment-roberta-large-english.xlsx"  # Replace with your desired Excel file name
data.to_excel(excel_file, index=False)